# Air Hockey Disk Collision Analysis

## Experimental Setup

- Two disks (4 inch diameter) on an air table moving in the XY plane
- Each disk equipped with an IMU chip (LSM6DS3) measuring acceleration and angular velocity
- Data collected via BLE at 50 Hz sampling rate
- Disk 1 is launched from the edge using a string/bow mechanism
- Disk 2 is initially stationary in the center

## Research Question

According to Newton's Third Law, the forces during collision should be equal and opposite:

$$F_1 = -F_2$$

Since the disks have equal mass ($m_1 = m_2$), the accelerations should be equal in magnitude:

$$|a_1| = |a_2|$$

**Is this observed in the experimental data?**

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

plt.style.use('seaborn-v0_8-darkgrid')
plt.rcParams['figure.figsize'] = [14, 5]
plt.rcParams['font.size'] = 11

## Dataset 1: Central Collision (1hits2-central.csv)

Disk 1 launched at ~3.64s, collides with Disk 2 at ~4.62s

In [ ]:
df1 = pd.read_csv('1hits2-central.csv')

# Calculate acceleration magnitude in XY plane
df1['a1_xy'] = np.sqrt(df1['ax1_g']**2 + df1['ay1_g']**2)
df1['a2_xy'] = np.sqrt(df1['ax2_g']**2 + df1['ay2_g']**2)

print(f"Duration: {df1['time_s'].max():.2f}s, {len(df1)} samples at 50 Hz")

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(14, 8))
fig.suptitle('Central Collision Analysis', fontsize=14, fontweight='bold')

# Full timeline
ax1 = axes[0, 0]
ax1.plot(df1['time_s'], df1['a1_xy'], 'b-', label='Disk 1 |a_xy|', linewidth=1)
ax1.plot(df1['time_s'], df1['a2_xy'], 'r-', label='Disk 2 |a_xy|', linewidth=1, alpha=0.8)
ax1.axvline(x=3.64, color='green', linestyle='--', alpha=0.7, label='Launch')
ax1.axvline(x=4.62, color='orange', linestyle='--', alpha=0.7, label='Collision')
ax1.set_ylabel('Acceleration (g)')
ax1.set_xlabel('Time (s)')
ax1.set_title('Full Timeline')
ax1.legend(loc='upper right')
ax1.set_ylim(0, 0.5)

# Collision zoom
ax2 = axes[0, 1]
collision = df1[(df1['time_s'] >= 4.4) & (df1['time_s'] <= 5.0)]
ax2.plot(collision['time_s'], collision['a1_xy'], 'b-o', label='Disk 1', linewidth=2, markersize=5)
ax2.plot(collision['time_s'], collision['a2_xy'], 'r-s', label='Disk 2', linewidth=2, markersize=5)
ax2.axvline(x=4.62, color='orange', linestyle='--', alpha=0.7)
ax2.set_ylabel('Acceleration (g)')
ax2.set_xlabel('Time (s)')
ax2.set_title('Collision Detail (4.4-5.0s)')
ax2.legend()

# Gyroscope
ax3 = axes[1, 0]
ax3.plot(df1['time_s'], df1['gz1_dps'], 'b-', label='Disk 1 ωz', linewidth=1)
ax3.plot(df1['time_s'], df1['gz2_dps'], 'r-', label='Disk 2 ωz', linewidth=1, alpha=0.8)
ax3.axvline(x=3.64, color='green', linestyle='--', alpha=0.7)
ax3.axvline(x=4.62, color='orange', linestyle='--', alpha=0.7)
ax3.set_ylabel('Angular velocity (°/s)')
ax3.set_xlabel('Time (s)')
ax3.set_title('Rotation (Gyro Z)')
ax3.legend()

# XY components at collision
ax4 = axes[1, 1]
ax4.plot(collision['time_s'], collision['ax1_g'], 'b-', label='Disk 1 ax', linewidth=1.5)
ax4.plot(collision['time_s'], collision['ay1_g'], 'b--', label='Disk 1 ay', linewidth=1.5)
ax4.plot(collision['time_s'], collision['ax2_g'], 'r-', label='Disk 2 ax', linewidth=1.5)
ax4.plot(collision['time_s'], collision['ay2_g'], 'r--', label='Disk 2 ay', linewidth=1.5)
ax4.axvline(x=4.62, color='orange', linestyle='--', alpha=0.7)
ax4.set_ylabel('Acceleration (g)')
ax4.set_xlabel('Time (s)')
ax4.set_title('X and Y Components')
ax4.legend(loc='upper right', fontsize=9)

plt.tight_layout()
plt.show()

In [ ]:
# Collision metrics
t_collision = 4.62
idx = df1[df1['time_s'] == t_collision].index[0]

print("=== Central Collision at t=4.62s ===")
print(f"Disk 1: |a| = {df1.loc[idx, 'a1_xy']:.4f} g")
print(f"Disk 2: |a| = {df1.loc[idx, 'a2_xy']:.4f} g")
print(f"Ratio |a1|/|a2| = {df1.loc[idx, 'a1_xy']/df1.loc[idx, 'a2_xy']:.2f}")
print(f"\nExpected ratio (Newton's 3rd Law): 1.00")
print(f"Observed ratio: 2.18 — NOT equal!")

## Dataset 2: Off-Center Collision (1hits2.csv)

A glancing blow that imparts significant spin to Disk 2

In [ ]:
df2 = pd.read_csv('1hits2.csv')

df2['a1_xy'] = np.sqrt(df2['ax1_g']**2 + df2['ay1_g']**2)
df2['a2_xy'] = np.sqrt(df2['ax2_g']**2 + df2['ay2_g']**2)

print(f"Duration: {df2['time_s'].max():.2f}s, {len(df2)} samples")

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(14, 8))
fig.suptitle('Off-Center Collision Analysis', fontsize=14, fontweight='bold')

# Full timeline
ax1 = axes[0, 0]
ax1.plot(df2['time_s'], df2['a1_xy'].clip(upper=3), 'b-', label='Disk 1', linewidth=1)
ax1.plot(df2['time_s'], df2['a2_xy'].clip(upper=3), 'r-', label='Disk 2', linewidth=1, alpha=0.8)
ax1.axvline(x=1.14, color='orange', linestyle='--', alpha=0.7, label='Collision')
ax1.set_ylabel('Acceleration (g)')
ax1.set_xlabel('Time (s)')
ax1.set_title('Full Timeline (clipped at 3g)')
ax1.legend()

# Collision zoom
ax2 = axes[0, 1]
collision = df2[(df2['time_s'] >= 0.9) & (df2['time_s'] <= 1.5)]
ax2.plot(collision['time_s'], collision['a1_xy'], 'b-o', label='Disk 1', linewidth=2, markersize=5)
ax2.plot(collision['time_s'], collision['a2_xy'], 'r-s', label='Disk 2', linewidth=2, markersize=5)
ax2.axvline(x=1.14, color='orange', linestyle='--', alpha=0.7)
ax2.set_ylabel('Acceleration (g)')
ax2.set_xlabel('Time (s)')
ax2.set_title('Collision Detail (0.9-1.5s)')
ax2.legend()

# Gyroscope - full
ax3 = axes[1, 0]
ax3.plot(df2['time_s'], df2['gz1_dps'], 'b-', label='Disk 1 ωz', linewidth=1)
ax3.plot(df2['time_s'], df2['gz2_dps'], 'r-', label='Disk 2 ωz', linewidth=1, alpha=0.8)
ax3.axvline(x=1.14, color='orange', linestyle='--', alpha=0.7)
ax3.set_ylabel('Angular velocity (°/s)')
ax3.set_xlabel('Time (s)')
ax3.set_title('Rotation — Note Disk 2 spin after collision')
ax3.legend()

# Gyroscope zoom
ax4 = axes[1, 1]
rot_region = df2[(df2['time_s'] >= 0.8) & (df2['time_s'] <= 2.0)]
ax4.plot(rot_region['time_s'], rot_region['gz1_dps'], 'b-o', label='Disk 1 ωz', linewidth=2, markersize=3)
ax4.plot(rot_region['time_s'], rot_region['gz2_dps'], 'r-s', label='Disk 2 ωz', linewidth=2, markersize=3)
ax4.axvline(x=1.14, color='orange', linestyle='--', alpha=0.7)
ax4.set_ylabel('Angular velocity (°/s)')
ax4.set_xlabel('Time (s)')
ax4.set_title('Rotation Detail — Disk 2 gains 37°/s')
ax4.legend()

plt.tight_layout()
plt.show()

In [ ]:
# Off-center collision metrics
idx = df2[df2['time_s'] == 1.14].index[0]

print("=== Off-Center Collision at t=1.14s ===")
print(f"Disk 1: |a| = {df2.loc[idx, 'a1_xy']:.4f} g")
print(f"Disk 2: |a| = {df2.loc[idx, 'a2_xy']:.4f} g")
print(f"Ratio |a1|/|a2| = {df2.loc[idx, 'a1_xy']/df2.loc[idx, 'a2_xy']:.2f}")

# Rotation change
before2 = df2[(df2['time_s'] >= 0.5) & (df2['time_s'] < 1.1)]['gz2_dps'].mean()
after2 = df2[(df2['time_s'] > 1.2) & (df2['time_s'] <= 1.8)]['gz2_dps'].mean()

print(f"\nDisk 2 rotation: {before2:.1f} → {after2:.1f} °/s (Δ = {after2-before2:.1f} °/s)")
print(f"\nThis is a glancing blow — Disk 2 received rotational impulse")

## Dataset 3: Another Collision (2.csv)

Launch at ~3.58s, disk-disk collision at ~3.94s, wall collision at ~4.58s

In [ ]:
df3 = pd.read_csv('2.csv')

df3['a1_xy'] = np.sqrt(df3['ax1_g']**2 + df3['ay1_g']**2)
df3['a2_xy'] = np.sqrt(df3['ax2_g']**2 + df3['ay2_g']**2)

print(f"Duration: {df3['time_s'].max():.2f}s, {len(df3)} samples")

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(14, 8))
fig.suptitle('Dataset 3 Analysis', fontsize=14, fontweight='bold')

# Full timeline
ax1 = axes[0, 0]
ax1.plot(df3['time_s'], df3['a1_xy'].clip(upper=4), 'b-', label='Disk 1', linewidth=1)
ax1.plot(df3['time_s'], df3['a2_xy'].clip(upper=4), 'r-', label='Disk 2', linewidth=1, alpha=0.8)
ax1.set_ylabel('Acceleration (g)')
ax1.set_xlabel('Time (s)')
ax1.set_title('Full Timeline (clipped at 4g)')
ax1.legend()

# Launch and collision region
ax2 = axes[0, 1]
region = df3[(df3['time_s'] >= 3.4) & (df3['time_s'] <= 4.8)]
ax2.plot(region['time_s'], region['a1_xy'].clip(upper=2), 'b-o', label='Disk 1', linewidth=2, markersize=4)
ax2.plot(region['time_s'], region['a2_xy'].clip(upper=2), 'r-s', label='Disk 2', linewidth=2, markersize=4)
ax2.axvline(x=3.58, color='green', linestyle='--', alpha=0.7, label='Launch')
ax2.axvline(x=3.94, color='orange', linestyle='--', alpha=0.7, label='Disk collision')
ax2.axvline(x=4.58, color='purple', linestyle='--', alpha=0.7, label='Wall hit')
ax2.set_ylabel('Acceleration (g)')
ax2.set_xlabel('Time (s)')
ax2.set_title('Events (3.4-4.8s)')
ax2.legend(fontsize=9)

# Gyroscope
ax3 = axes[1, 0]
ax3.plot(df3['time_s'], df3['gz1_dps'], 'b-', label='Disk 1 ωz', linewidth=1)
ax3.plot(df3['time_s'], df3['gz2_dps'], 'r-', label='Disk 2 ωz', linewidth=1, alpha=0.8)
ax3.axvline(x=3.58, color='green', linestyle='--', alpha=0.7)
ax3.axvline(x=3.94, color='orange', linestyle='--', alpha=0.7)
ax3.set_ylabel('Angular velocity (°/s)')
ax3.set_xlabel('Time (s)')
ax3.set_title('Rotation')
ax3.legend()

# Rotation detail
ax4 = axes[1, 1]
rot = df3[(df3['time_s'] >= 3.8) & (df3['time_s'] <= 4.2)]
ax4.plot(rot['time_s'], rot['gz1_dps'], 'b-o', label='Disk 1 ωz', linewidth=2, markersize=4)
ax4.plot(rot['time_s'], rot['gz2_dps'], 'r-s', label='Disk 2 ωz', linewidth=2, markersize=4)
ax4.axvline(x=3.94, color='orange', linestyle='--', alpha=0.7)
ax4.set_ylabel('Angular velocity (°/s)')
ax4.set_xlabel('Time (s)')
ax4.set_title('Rotation at Collision — Disk 2 jumps to 50°/s')
ax4.legend()

plt.tight_layout()
plt.show()

In [ ]:
# Dataset 3 collision analysis
idx = df3[df3['time_s'] == 3.94].index[0]

print("=== Disk-Disk Collision at t=3.94s ===")
print(f"Disk 1: |a| = {df3.loc[idx, 'a1_xy']:.3f} g")
print(f"Disk 2: |a| = {df3.loc[idx, 'a2_xy']:.3f} g")

print(f"\nDisk 2 rotation before: ~0 °/s")
print(f"Disk 2 rotation after: ~50 °/s")
print(f"\nThis is a tangential collision — Disk 2 mostly spins, barely translates")

## Conclusion: Why Accelerations Are Not Equal

### The Physics is Correct

Newton's Third Law states that **forces** are equal and opposite:
$$\vec{F}_1 = -\vec{F}_2$$

For equal masses, this means **accelerations should be equal in magnitude**:
$$|\vec{a}_1| = |\vec{a}_2|$$

### The Problem: Temporal Resolution

At **50 Hz sampling** (20 ms between samples), the collision duration (~1-5 ms) is **shorter than one sample period**.

This means:
- We capture at most **one data point** during the collision
- Each sensor samples at a slightly different phase of the impulse curve
- One sensor might catch the peak, another the rising/falling edge
- The measured accelerations appear unequal even though the true impulse is symmetric

### Solution: Increase Sampling Rate

Upgrade from 50 Hz to **500 Hz**:
- 10-25 data points per collision instead of 1
- LSM6DS3 supports up to 833 Hz or 1.66 kHz
- Will capture the full impulse profile
- Can verify $|a_1| = |a_2|$ at the true peaks

### Firmware Changes Required

```cpp
// Increase sensor rate
imu.settings.accelSampleRate = 833;
imu.settings.gyroSampleRate = 833;

// Decrease sample interval
#define SAMPLE_INTERVAL_MS 2  // was 20

// Increase buffer size (10x more data)
#define MAX_SAMPLES 5000  // was 500
```